In [1]:
manufacturer = 'AIRTEX'
type_code = '2208'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()
list_part_number = df['Part_Number'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

import numpy as np

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url, part_number in zip(list_url, list_part_number):
    work.put_nowait((url, part_number))

def crawler():
    global df
    
    while not work.empty():
        url, part_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                resp = requests.get(url,
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                if len(html.xpath('//span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]')) != 0:
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             'Manufacturer': manufacturer,
                                             'Part_Number': part_number,
                                             'Url': url}])
                    
                    status = 'ok'
                    
                    break

                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =
                
                list_part_number = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]')[0].text for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                number_index = np.where(np.array(list_part_number) == part_number)[0]
                
                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]')[0].text for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                manufacturer_index = np.where(np.array(list_manufacturer) == manufacturer)[0]
                
                # = = = = = = = = = = = = = = =
                
                index = list(np.intersect1d(number_index, manufacturer_index))
                
                # = = = = = = = = = = = = = = =
                
                if len(index) == 1:
                    index = index[0]
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_span = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]') for _ in list_part]
                    for _ in list_span:
                        note_1 = _[0].xpath('./span[contains(@class, "span-link-underline-remover")]/text()')
                        if len(note_1) == 0:
                            note_1 = ''
                        else:
                            note_1 = note_1[0].strip()
   
                    # = = = = = = = = = = = = = = =
                    
                    list_span = [_.xpath('./descendant::div[contains(@class, "listing-text-row") and not(contains(@class, "moreinfo-truck"))]') for _ in list_part]
                    for _ in list_span:
                        note_2 = _[0].xpath('./span/span/text()')
                        if len(note_2) == 0:
                            note_2 = ''
                        else:
                            note_2 = [_.strip() for _ in note_2]
                            while '' in note_2:
                                note_2.remove('')
                            note_2 = '\n'.join(note_2)  

                    # = = = = = = = = = = = = = = =
                    
                    list_span = [_.xpath('./descendant::span[contains(@title, "Replaces these Alternate/ OE Part Numbers")]') for _ in list_part]
                    for _ in list_span:
                        if len(_) == 0:
                            oe = ''
                        else:
                            oe = _[0].xpath('./text()')[0].replace(', ', ';')
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_span = [_.xpath('./descendant::img[contains(@id, "inlineimg_thumb")]') for _ in list_part]
                    for _ in list_span:
                        if len(_) == 0:
                            src = ''
                        else:
                            src = 'https://www.rockauto.com' + _[0].xpath('./@src')[0].replace('__ra_m.', '.')
     
                    # = = = = = = = = = = = = = = =
                    
                    list_span = [_.xpath('./descendant::input[contains(@id, "listing_data_essential")]') for _ in list_part]
                    for _ in list_span:
                        if len(_) == 0:
                            part_key = ''
                        else:
                            part_key = json.loads(_[0].xpath('./@value')[0])['partkey']
                            
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             '序号': '',
                                             'Manufacturer': manufacturer,
                                             'Part_Number': part_number,
                                             'Vehcile': '',
                                             'Alternate_OE_Part_Numbers': oe,
                                             'Note_1': note_1,
                                             'Note_2': note_2,
                                             'Pic': '',
                                             'Url': url,
                                             'Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + type_code + '&pk=' + part_key,
                                             'Src': src,
                                             'Type_Code': type_code,
                                             'Part_Key': part_key,}])
                
                    status = 'ok'
                
                    break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Manufacturer': manufacturer,
                                     'Part_Number': part_number,
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：351

AW1069  <->  [ok] - 剩余数量：336
AW2069  <->  [ok] - 剩余数量：335
AW2050  <->  [ok] - 剩余数量：334
AW1120H  <->  [ok] - 剩余数量：333
AW2047  <->  [ok] - 剩余数量：332
AW1127  <->  [ok] - 剩余数量：331
AW3413  <->  [ok] - 剩余数量：330
AW3414  <->  [ok] - 剩余数量：329
AW1028H  <->  [ok] - 剩余数量：328
AW1099  <->  [ok] - 剩余数量：327
AW3713  <->  [ok] - 剩余数量：326
AW2063  <->  [ok] - 剩余数量：325
AW3717  <->  [ok] - 剩余数量：324
AW3410  <->  [ok] - 剩余数量：323
AW3411  <->  [ok] - 剩余数量：322
AW4011  <->  [ok] - 剩余数量：321
AW4002  <->  [ok] - 剩余数量：320
AW4014  <->  [ok] - 剩余数量：319
AW4035  <->  [ok] - 剩余数量：318
AW4037  <->  [ok] - 剩余数量：317
AW4051  <->  [ok] - 剩余数量：316
AW4023  <->  [ok] - 剩余数量：315
AW4039  <->  [ok] - 剩余数量：314
AW4055  <->  [ok] - 剩余数量：313
AW4034  <->  [ok] - 剩余数量：312
AW4024  <->  [ok] - 剩余数量：311
AW4056  <->  [ok] - 剩余数量：310
AW4049  <->  [ok] - 剩余数量：309
AW4022  <->  [ok] - 剩余数量：308
AW4057  <->  [ok] - 剩余数量：307
AW4065  <->  [ok] - 剩余数量：306
AW4082  <->  [ok] - 剩余数量：305
AW4069  <->  [ok] - 剩余数量：304
AW4078  <->  [ok] - 剩余数量：303
AW4

/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_26098/4180399349.py:94: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  number_index = np.where(np.array(list_part_number) == part_number)[0]
/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_26098/4180399349.py:102: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  manufacturer_index = np.where(np.array(list_manufacturer) == manufacturer)[0]


AW4128  <->  [ok] - 剩余数量：279
AW4133  <->  [ok] - 剩余数量：278
AW455  <->  [ok] - 剩余数量：277
AW4125  <->  [ok] - 剩余数量：276
AW5000P  <->  [ok] - 剩余数量：275
AW4053  <->  [ok] - 剩余数量：274
AW4127  <->  [ok] - 剩余数量：273
AW5016H  <->  [ok] - 剩余数量：272
AW5014P  <->  [ok] - 剩余数量：271
AW5036  <->  [ok] - 剩余数量：270
AW494  <->  [ok] - 剩余数量：269
AW5037  <->  [ok] - 剩余数量：268
AW5030  <->  [ok] - 剩余数量：267
AW5038H  <->  [ok] - 剩余数量：266
AW5006  <->  [ok] - 剩余数量：265
AW5040  <->  [ok] - 剩余数量：264
AW5029  <->  [ok] - 剩余数量：263
AW5048  <->  [ok] - 剩余数量：262
AW5022  <->  [ok] - 剩余数量：261
AW5044H  <->  [ok] - 剩余数量：260
AW5050N  <->  [ok] - 剩余数量：259
AW5072  <->  [ok] - 剩余数量：258
AW5070  <->  [ok] - 剩余数量：257
AW5043  <->  [ok] - 剩余数量：256
AW5076  <->  [ok] - 剩余数量：255
AW5078  <->  [ok] - 剩余数量：254
AW5063  <->  [ok] - 剩余数量：253
AW5079  <->  [ok] - 剩余数量：252
AW4089  <->  [ok] - 剩余数量：251
AW5085  <->  [ok] - 剩余数量：250
AW5081  <->  [ok] - 剩余数量：249
AW5073  <->  [ok] - 剩余数量：248
AW5094  <->  [ok] - 剩余数量：247
AW5098  <->  [ok] - 剩余数量：246
AW6000  <-

AW981  <->  [ok] - 剩余数量：0
AW9140  <->  [ok] - 剩余数量：0
AW9493  <->  [ok] - 剩余数量：0
AW9320  <->  [ok] - 剩余数量：0
AW9305  <->  [ok] - 剩余数量：0
AW6242  <->  [ok] - 剩余数量：0
AW9272  <->  [ok] - 剩余数量：0

爬虫输出中

爬虫结束
